In [4]:
"""
This notebook generates data files that contain the counts we would measure for:
state with 'qubit_number'
in the basis 'letters'
for a number of 'runs'

It writes the simulated data in a file in path ".../Tomography/StateTomoData/Simulated_GHZ"
(that you might need to create)
This data is written in a format such that it can be analysed with the 'Quantum State Tomography'
jupyter notebook in '.../Tomography'
"""


'\nThis notebook generates data files that contain the counts we would measure for:\nstate with \'qubit_number\'\nin the basis \'letters\'\nfor a number of \'runs\'\n\nIt writes the simulated data in a file in path ".../Tomography/StateTomoData/Simulated_GHZ"\n(that you might need to create)\nThis data is written in a format such that it can be analysed with the \'Quantum State Tomography\'\njupyter notebook in \'.../Tomography\'\n'

In [5]:
import numpy as np
import os
import glob
from pathlib import Path

In [6]:
working_dir = Path(os.getcwd()).parent
os.chdir(working_dir)

In [8]:
from tomography import *
from nestedforloop import get_iterator
from efficiencies import get_channels_eff, set_raw_counts
from optimization import *
from constants import *
from pauliexpectations import *

In [11]:
working_dir_data = os.getcwd()+'\StateTomoData'
os.chdir(working_dir_data)

In [12]:
### Defining the params
qubit_number=4
runs=100000

### 4 qubit GHZ ###
state=(np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])+np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]))/np.sqrt(2)

### 2 qubit Bell state ###
# state=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)

Counts_file = np.zeros((3**qubit_number,2**qubit_number))

In [13]:
def get_proj_iterator(N_qubit, basis):
    d={"x": [[1,1]/np.sqrt(2),
             [1,-1]/np.sqrt(2)],
       "y":[[1,1j]/np.sqrt(2),
            [1,-1j]/np.sqrt(2)],
       "z":[[1,0],[0,1]],
       "a":[[0,1],[1,0]]}
    
    basis_order=get_iterator(len(basis), N_qubit)
    projectors_order=get_iterator(2, N_qubit)
    columns=[]
    
    character_matrix=[]
    for i in basis_order:
        string=""
        for j in i:
            string=string+basis[j]
        lines=[]    
        for k in projectors_order:
            o=0
            vect=[]
            for n in k:
                if o > 0:
                    vect=np.kron(vect,d[string[o]][n])
                else:
                    vect=d[string[o]][n]
                o=o+1
            lines.append(vect)    
        character_matrix.append(lines)         
        columns.append(string)   
        
    return character_matrix

In [17]:
######################################################
#----- CREATING THE FILES FOR AND THE COUNTS -------
######################################################
print('Creating the counts and files')
letters = ['x', 'y', 'z']
bases = []

projector_matrix = get_proj_iterator(qubit_number, letters)

for i in range(3**qubit_number):
    for j in range(2**qubit_number):
        Counts_file[i][j] = runs*abs(np.trace(np.dot(np.outer(projector_matrix[i][j],projector_matrix[i][j]),np.outer(state,state))))  

for i in letters:
    if qubit_number == 1:
        base = i 
        bases.append(base)
    for j in letters:
        if qubit_number == 2:
            base = i + j
            bases.append(base)
        for k in letters:
            if qubit_number == 3:
                base = i + j + k 
                bases.append(base)
            for l in letters:
                if qubit_number == 4:
                    base = i + j + k + l
                    bases.append(base)

                    
for i in range(len(bases)):
    f = open(working_dir_data+"\Simulated\GHZ\Bigiteration_0_" + str(bases[i]) + ".txt", "a")
    f.truncate(0)
    for j in range(2**qubit_number):
        f.write(str(Counts_file[i,j])+' ')
        
        
    ### Considering double emission
    p=0.00001
    if qubit_number==1:
        double_emission=[p*runs]*1
    elif qubit_number==2:
        double_emission=[p*runs]*5
    elif qubit_number==3:
        double_emission=[p*runs]*7
    elif qubit_number==4:
        double_emission=[p*runs]*64
        
    for j in range(len(double_emission)):
        f.write(str(double_emission[j])+' ')
    f.close()

Creating the counts and files


In [23]:
######################################################
#----- CREATING THE FILES FOR THE EFFICIENCIES -------
######################################################

print('Creating the counts and files for the efficiency')
letters = ['z', 'a']
bases = []

projector_matrix2 = get_proj_iterator(qubit_number, letters)

for i in range(2**qubit_number):
    for j in range(2**qubit_number):
        Counts_file[i][j] = runs*abs(np.trace(np.dot(np.outer(projector_matrix2[i][j],projector_matrix2[i][j]),np.outer(state,state))))  

for i in letters:
    if qubit_number == 1:
        base = i 
        bases.append(base)
    for j in letters:
        if qubit_number == 2:
            base = i + j
            bases.append(base)
        for k in letters:
            if qubit_number == 3:
                base = i + j + k 
                bases.append(base)
            for l in letters:
                if qubit_number == 4:
                    base = i + j + k + l
                    bases.append(base)
                    
for i in range(len(bases)):
    f = open(working_dir_data+"/Simulated/GHZ/Bigiteration_0_" + str(bases[i]) + ".txt", "a")
    f.truncate(0)
    for j in range(2**qubit_number):
        f.write(str(Counts_file[i,j]))
        f.write(' ')
    f.close()

Creating the counts and files for the efficiency
